In [1]:
!pip install pycryptodome -q

In [30]:
import numpy as np
from db_lib import db
from aes_encrypt import AES_Encrypt
from Crypto.Cipher import AES
import hashlib

In [3]:
# Connect to database and get cursor
cursor = db()

# Write Query
sql = 'SELECT * FROM clinical_document.q_document'
# Run Query
data = cursor.sql(sql)
# Close Connection
del cursor


key = AES.get_random_bytes(32)
key

b'\xa141,#\xe2;\xcbGF\xdb\xa3\xd3\xe3\xbc\x80\xff\rc\xaf\xc0n\xd2\xf9\xf5lk\xbbs\xeb\x05\xc0'

In [4]:
# Connect to database and get cursor
cursor = db()

tag = 'mrn'
attribute = 'value'
sql = f"SELECT DISTINCT uuid FROM \
(SELECT jsonb_array_elements(attr)->>'uuid' as uuid, jsonb_array_elements(attr)->>'tag' as tag, jsonb_array_elements(attr)::jsonb as attributes \
FROM clinical_document.document, jsonb_array_elements(attr) with ordinality arr(item_object, position) ) tmp \
WHERE tag = '{tag}' \
AND attributes->tag ? '{attribute}'"

uuids = np.asarray(cursor.sql(sql)).flatten()

# loop Through uuids
for uuid in uuids:
    sql = f"SELECT md5, item_object \
            FROM clinical_document.document, jsonb_array_elements(attr) with ordinality arr(item_object, position) \
            WHERE arr.item_object->>'uuid' = '1f4aa6b8-0356-433f-b8a2-7971a158bdb7';"
    data = np.array(cursor.sql(sql)).T
    md5 = data[0][0]
    print(f"{md5=}")
    json_obj = data[1][0]
    print(json_obj)
    value = json_obj[f'{tag}'][f'{attribute}'].encode("utf8")
    print(value)
    
    #aes = AES_Encrypt(key=key)
    #cipher_text, tag_, nonce = aes.encrypt_text(value)
    #print(cipher_text, tag_, nonce)
    
    
    
    value = aes.decrypt_text(cipher_text, tag_, nonce)
    print(value.decode())
    
    del aes
    
    
    
print(uuids)
# Close Connection
del cursor

IndexError: index 0 is out of bounds for axis 0 with size 0

In [29]:
mrn = '333333'.encode("utf8")
# Encrypt
aes = AES_Encrypt(key=key)
cipher_text, cipher_tag, nonce = aes.encrypt_text(mrn)
del aes
print(cipher_text.hex())

b3e5fff4d458


In [34]:
hashlib.sha256(mrn).hexdigest()

'68487dc295052aa79c530e283ce698b8c6bb1b42ff0944252e1910dbecdc5425'